In [ ]:
!nvidia-smi


In [ ]:
import tensorflow as tf

import os
import time

from matplotlib import pyplot as plt
from IPython import display
import random
import numpy as np
import cv2
import scipy.ndimage

In [ ]:
'''!pip install -U tensorboard'''

## Load the dataset

You can download this dataset and similar datasets from [here](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets). As mentioned in the [paper](https://arxiv.org/abs/1611.07004) we apply random jittering and mirroring to the training dataset.

* In random jittering, the image is resized to `286 x 286` and then randomly cropped to `256 x 256`
* In random mirroring, the image is randomly flipped horizontally i.e left to right.

In [ ]:
'''_URL = "https://storage.googleapis.com/kaggle-data-sets/29561%2F37705%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1597964357&Signature=WTIkK4OTmmEGu4Sy5J4knVRxKccFCWDADrWOxnqJUd1B6ZCQbcM2kUv8%2FSKTANffZTT2a3gywPbuKBfD1QqfB%2BoUhnzhqNvU19cWPCL5zNpIUmnCS7Sj%2F8ZeLjaARuAmh5WdB%2FiCso2x6toc9QunUgbsgf6U984xGOZjoeLB59LkFGV%2B0EIvaaqpZ%2Bo9D1xcO%2FTTiDR5vElA9SV6t7UYusaKe585CQ6RbDYgLwxoaMBm3AYGIpHdWSxK3Lmbu4nj%2BmuEKg1YjvNO%2F4asUipd6GrZZLJbWPi%2B0uSUKCpi5qPHnEZCXjWMjEX4LTHg20PSu0MPpKXJZH8ZIUpMs1qYOQ%3D%3D"
path_to_zip = tf.keras.utils.get_file('img_align_celeba.tar.gz',
                                      origin=_URL,
                                      extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'img_align_celeba/img_align_celeba/')'''

In [ ]:
'''print(PATH)
'''

In [ ]:
"""!ls  /root/.keras/datasets/images/"""

In [ ]:
PATH="/content/drive/My Drive/grad_project/approach2/NoBackground_Dataset/"

In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 1 
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  image = tf.cast(image, tf.float32)

  return image

In [ ]:
'''img = load(PATH+'101301.jpg')
# casting to int for matplotlib to show the image
plt.figure()
plt.imshow(img/255.0)'''

In [ ]:
'''image = (img / 127.5) - 1
ans = img2sketch_canny(image)
plt.figure()
plt.imshow(ans[:,:,0],cmap="gray")'''

In [ ]:
def resize(image, height, width):
  image = tf.image.resize(image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return image

In [ ]:
'''def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]'''

In [ ]:
# normalizing the images to [-1, 1]

def normalize(image):
  image = (image / 127.5) - 1

  return image

In [ ]:
@tf.function()
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = resize(image, IMG_WIDTH, IMG_HEIGHT)

  # randomly cropping to 256 x 256 x 3
  #input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    image = tf.image.flip_left_right(image)

  return image

As you can see in the images below
that they are going through random jittering
Random jittering as described in the paper is to

1. Resize an image to bigger height and width
2. Randomly crop to the target size
3. Randomly flip the image horizontally

In [ ]:
'''plt.figure(figsize=(6, 6))
for i in range(4):
  inp = random_jitter(img)
  plt.subplot(2, 2, i+1)
  plt.imshow(inp/255.0)
  plt.axis('off')
plt.show()'''

In [ ]:
def load_image_train(image_file):
  image = load(image_file)
  image = random_jitter(image)
  image = normalize(image)

  return image

In [ ]:
def load_image_test(image_file):
  image = load(image_file)
  image = resize(image, IMG_HEIGHT, IMG_WIDTH)
  image = normalize(image)

  return image

In [ ]:
'''

def img2edgepair (image):
  image = image.astype(np.uint8)
  #edge = cv2.rotate(image, cv2.cv2.ROTATE_90_CLOCKWISE)

  edge = cv2.Canny(image,100,200)
  #edge_expand_dim = np.expand_dims(edge,axis=-1)
  #edge_copy_dim = np.repeat(examp_add_dim,3,axis=2)
  #print(edge.shape)
  #print(image.shape)

  return image,edge

@tf.function(input_signature=[tf.TensorSpec(None, tf.float32)])
def img2edgepair_numpy(input):
  img1 = tf.numpy_function(  func=img2edgepair , inp=[input]  , Tout=(tf.uint8 , tf.uint8) )
  return img1'''

In [ ]:
def img2sketch_canny(img):
  #input shape is (width,height,channels)
  #output shape is (1,width,height)
  img=img.numpy()
  img = (img*0.5+0.5)*255 # un-normalize image and put it in 0-255 range
  img = img.astype(np.uint8)
  img = np.reshape(img,
  (img.shape[-3],img.shape[-2],img.shape[-1]))
  sketch = cv2.Canny(img,100,200)
  sketch = sketch.astype(np.float32)
  sketch = normalize(sketch)
  return np.expand_dims(sketch,-1)

In [ ]:
'''print(img.shape)
a = img2sketch_canny(img/255.0)
print(a.shape)
plt.figure()
plt.imshow(a[...,0]*0.5+0.5)
plt.figure()
plt.imshow(img/255)'''

## Input Pipeline

In [ ]:
# we have 2222 images ... use 2000 for the training dataset and 222 for the test dataset
train_dataset = tf.data.Dataset.list_files(PATH+"train/"+"*.png")
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
#train_dataset = train_dataset.map( img2edgepair_numpy )
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset =  tf.data.Dataset.list_files(PATH+"val/"+"*.png")
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
'''x=random.randint(0,1000)
x=3
example = iter(train_dataset)
for i in range(x-1):
  next(example)
example=np.array(next(example))
print("..................................")
print(example.shape)
#print(example[0])
plt.imshow((example[0]+1)*127.5/255)
print("picture size : ",example[0].shape)'''

In [ ]:
'''sketch = img2sketch((example[0]+1)*127.5/255)
print(sketch.shape)
plt.imshow(sketch[...,0],cmap="gray")'''

## Build the Generator
  * The architecture of generator is a modified U-Net.
  * Each block in the encoder is (Conv -> Batchnorm -> Leaky ReLU)
  * Each block in the decoder is (Transposed Conv -> Batchnorm -> Dropout(applied to the first 3 blocks) -> ReLU)
  * There are skip connections between the encoder and decoder (as in U-Net).


In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
'''down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)'''

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
'''up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)'''

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,1])
  noise_input = tf.keras.layers.Input(shape=[256,256,1])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  #x = tf.keras.layers.Multiply()([inputs, noise_input])
  x=tf.keras.layers.Concatenate()([inputs, noise_input])
  #x= inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=[inputs,noise_input], outputs=x)

In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
'''sketch = img2sketch(inp)
print(sketch[tf.newaxis,...].shape)
plt.imshow(sketch[...,0],cmap="gray")
gen_output = generator(sketch[tf.newaxis,...], training=False)
plt.figure()
plt.imshow(gen_output[0,...])'''

* **Generator loss**
  * It is a sigmoid cross entropy loss of the generated images and an **array of ones**.
  * The [paper](https://arxiv.org/abs/1611.07004) also includes L1 loss which is MAE (mean absolute error) between the generated image and the target image.
  * This allows the generated image to become structurally similar to the target image.
  * The formula to calculate the total generator loss = gan_loss + LAMBDA * l1_loss, where LAMBDA = 100. This value was decided by the authors of the [paper](https://arxiv.org/abs/1611.07004).

The training procedure for the generator is shown below:

In [ ]:
LAMBDA = 100

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l2_loss = tf.reduce_mean(tf.math.square(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l2_loss)

  return total_gen_loss, gan_loss, l2_loss

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)


## Build the Discriminator
  * The Discriminator is a PatchGAN.
  * Each block in the discriminator is (Conv -> BatchNorm -> Leaky ReLU)
  * The shape of the output after the last layer is (batch_size, 30, 30, 1)
  * Each 30x30 patch of the output classifies a 70x70 portion of the input image (such an architecture is called a PatchGAN).
  * Discriminator receives 2 inputs.
    * Input image and the target image, which it should classify as real.
    * Input image and the generated image (output of generator), which it should classify as fake.
    * We concatenate these 2 inputs together in the code (`tf.concat([inp, tar], axis=-1)`)

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
'''disc_out = discriminator([sketch[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()'''

**Discriminator loss**
  * The discriminator loss function takes 2 inputs; **real images, generated images**
  * real_loss is a sigmoid cross entropy loss of the **real images** and an **array of ones(since these are the real images)**
  * generated_loss is a sigmoid cross entropy loss of the **generated images** and an **array of zeros(since these are the fake images)**
  * Then the total_loss is the sum of real_loss and the generated_loss


In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

The training procedure for the discriminator is shown below.

To learn more about the architecture and the hyperparameters you can refer the [paper](https://arxiv.org/abs/1611.07004).

![Discriminator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/dis.png?raw=1)


## Define the Optimizers and Checkpoint-saver


In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = '/content/drive/My Drive/grad_project/approach2/Training_checkpoints_4'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator,
                                 )

In [ ]:
#restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(checkpoint_dir+"/ckpt-28")

## Generate Images

Write a function to plot some images during training.

* We pass images from the test dataset to the generator.
* The generator will then translate the input image into the output.
* Last step is to plot the predictions and **voila!**

Note: The `training=True` is intentional here since
we want the batch statistics while running the model
on the test dataset. If we use training=False, we will get
the accumulated statistics learned from the training dataset
(which we don't want)

In [ ]:
def generate_images(model, test_input, tar):
  z = np.random.normal(0, 1, (1,256, 256,1))
  prediction = model([test_input,z], training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0,...,0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow( ((display_list[i] +1 )* 127.5/255),cmap="gray")
    plt.axis('off')
  plt.show()

In [ ]:
for  example_target in test_dataset.take(1):
  example_input = img2sketch_canny(example_target)
  print(example_input.shape)
  generate_images(generator, example_input[tf.newaxis,...], example_target)

## Training

* For each example input generate an output.
* The discriminator receives the input_image and the generated image as the first input. The second input is the input_image and the target_image.
* Next, we calculate the generator and the discriminator loss.
* Then, we calculate the gradients of loss with respect to both the generator and the discriminator variables(inputs) and apply those to the optimizer.
* Then log the losses to TensorBoard.

In [ ]:
EPOCHS = 150

In [ ]:
import datetime
#log_dir="./logs/"

'''summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))'''

###########################

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
disc_log_dir = './log/gradient_tape/' + current_time + '/discriminator'
gen_log_dir = './log/gradient_tape/' + current_time + '/generator'
disc_summary_writer = tf.summary.create_file_writer(disc_log_dir)
gen_summary_writer = tf.summary.create_file_writer(gen_log_dir)

In [ ]:
@tf.function
def train_step(input_image, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    #print(input_image.shape)
    #print(target.shape)
    z = np.random.normal(0, 1, (1,256, 256,1))
    #print("i am here : ",z.shape)
    #print("i am still here :",input_image.shape)
    gen_output = generator([input_image,z], training=True)

    disc_real_output = discriminator([input_image, target[...,0:3]], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target[...,0:3])
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with disc_summary_writer.as_default():
    #tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    #tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_gen_loss', disc_loss, step=epoch)
  with gen_summary_writer.as_default():
    tf.summary.scalar('disc_gen_loss', gen_total_loss, step=epoch)
    #
    #

The actual training loop:

* Iterates over the number of epochs.
* On each epoch it clears the display, and runs `generate_images` to show it's progress.
* On each epoch it iterates over the training dataset, printing a '.' for each example.
* It saves a checkpoint every 20 epochs.

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(140,epochs,1):
    start = time.time()

    #display.clear_output(wait=True)

    for example_target in test_ds.take(1):
      example_input=img2sketch_canny(example_target[tf.newaxis,...])
      generate_images(generator, example_input[tf.newaxis], example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (target) in train_ds.enumerate():
      input_image=img2sketch_canny(target)
      if(n%10==0):
        print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image[tf.newaxis,...], target, epoch)
    print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

This training loop saves logs you can easily view in TensorBoard to monitor the training progress. Working locally you would launch a separate tensorboard process. In a notebook, if you want to monitor with TensorBoard it's easiest to launch the viewer before starting the training.

To launch the viewer paste the following into a code-cell:

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Now run the training loop:

In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

In [ ]:
path_to_output = "/content/model_output_path/"

In [ ]:
'''!zip -r /content/model_output_path /content/model_output_path'''

In [ ]:
# Run the trained model on a few examples from the test dataset
img_name=0
for tar in test_dataset:
  img_name+=1
  inp=img2sketch_canny(tar[tf.newaxis,...])
  output = generator(inp[tf.newaxis,...] )
  output = output[0]*0.5+0.5
  plt.axis('off')
  plt.imshow(output)
  plt.savefig(path_to_output+str(img_name),transparent=True,bbox_inches='tight')
  #generate_images(generator, inp[tf.newaxis], tar)

In [ ]:
# Run the trained model on the test dataset
counter=0
for tar in test_dataset:
  counter+=1
  inp=img2sketch_canny(tar[tf.newaxis,...])
  generate_images(generator, inp[tf.newaxis], tar)
print(counter)

If you want to share the TensorBoard results _publicly_ you can upload the logs to [TensorBoard.dev](https://tensorboard.dev/) by copying the following into a code-cell.

Note: This requires a Google account.

```
!tensorboard dev upload --logdir  {log_dir}
```

Caution: This command does not terminate. It's designed to continuously upload the results of long-running experiments. Once your data is uploaded you need to stop it using the "interrupt execution" option in your notebook tool.

You can view the [results of a previous run](https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw) of this notebook on [TensorBoard.dev](https://tensorboard.dev/).

TensorBoard.dev is a managed experience for hosting, tracking, and sharing ML experiments with everyone.

It can also included inline using an `<iframe>`:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

Interpreting the logs from a GAN is more subtle than a simple classification or regression model. Things to look for::

* Check that neither model has "won". If either the `gen_gan_loss` or the `disc_loss` gets very low it's an indicator that this model is dominating the other, and you are not successfully training the combined model.
* The value `log(2) = 0.69` is a good reference point for these losses, as it indicates a perplexity of 2: That the discriminator is on average equally uncertain about the two options.
* For the `disc_loss` a value below `0.69` means the discriminator is doing better than random, on the combined set of real+generated images.
* For the `gen_gan_loss` a value below `0.69` means the generator i doing better than random at foolding the descriminator.
* As training progresses the `gen_l1_loss` should go down.

## Restore the latest checkpoint and test

In [ ]:
from PIL import Image
import os
import glob

In [ ]:
!ls {checkpoint_dir}

In [ ]:

checkpoint_dir="/content/drive/My Drive/grad_project/approach2/Training_checkpoints_4"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
#restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(checkpoint_dir+"/ckpt-30")

In [ ]:
  for j,example_target in enumerate(test_dataset.take(1)):
      example_input = img2sketch_canny(example_target[tf.newaxis,...])
      z = np.random.normal(0, 1, (1,256, 256,1))
      prediction = generator([example_input[tf.newaxis],z], training=True)
      l2_loss = tf.reduce_mean(tf.math.square(example_target[...,0:3] - prediction))
      #save predictions to a folder
      im=tf.keras.preprocessing.image.array_to_img(prediction[0,...]*0.5+0.5)
      #im=Image.fromarray(prediction*0.5+0.5)
      #im.save(newpath+"/"+str(j)+".png")

In [ ]:
print(l2_loss)

In [ ]:
path_to_ckpt = glob.glob(checkpoint_dir+"/"+"*.index")
print(path_to_ckpt)
print(len(path_to_ckpt))
path_to_save="/content/drive/My Drive/grad_project/approach2/output1.1/"

In [ ]:
for i,ckpt in enumerate(path_to_ckpt):
  print("checkpoint#",i+1,"...")
  #creat a folder to contain all checkpoint's images : 
  newpath = path_to_save + str(i)
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  # restoring a checkpoint in checkpoint_dir
  checkpoint.restore(ckpt[0:-6])
  for j,example_target in enumerate(test_dataset.take(3000)):
      example_input = img2sketch_canny(example_target[tf.newaxis,...])
      z = np.random.normal(0, 1, (1,256, 256,1))
      prediction = generator([example_input[tf.newaxis],z], training=True)
      #save predictions to a folder
      im=tf.keras.preprocessing.image.array_to_img(prediction[0,...]*0.5+0.5)
      #im=Image.fromarray(prediction*0.5+0.5)
      im.save(newpath+"/"+str(j)+".png")
      


## Generate using test dataset

In [ ]:
# Run the trained model on a few examples from the test dataset
for inp, tar in test_dataset.take(5):
  generate_images(generator, inp, tar)